<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [14]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}

# 전체 과정 사이클

In [17]:
url = "https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [20]:
base_df = pd.read_csv("/content/drive/MyDrive/KB인턴관련/crawl_1_euckr.csv", encoding='euc-kr')
base_df.tail()

,category,lawName,ENG,KOR
68167,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 7 (Demand for Materials Related to Con...,제7조(계약에 관한 자료 등의 요구)\n① 기획재정부장관은 정부조달협정가입국으로부터...
68168,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,Article 8 (Reporting Results of Contracts)\n(1...,제8조(계약실적보고)\n① 특례규정 제25조제1항에 따른 계약실적 보고서의 내용은 ...
68169,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제358호, 2013.8.7.>\n이 규칙은 공포한 날부터 시행한다."
68170,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDA <Ordinance of the Ministry of Strategy...,"부칙 <제360호, 2013.9.17.> (전자조달의 이용 및 촉진에 관한 법률 시..."
68171,재정ㆍ경제일반,특정조달을 위한 국가를 당사자로 하는 계약에 관한 법률 시행 특례규칙,ADDENDUM <Ordinance of the Ministry of Strateg...,"부칙 <제492호, 2015.7.20.>\n이 규칙은 정부조달에 관한 협정 개정 의..."


In [18]:
def tab_move():
    driver.switch_to.window(driver.window_handles[-1])
    ko_url = driver.current_url.replace("ENG", "KOR")
    driver.get(ko_url)
    driver.switch_to.frame("lawViewContent")

def tab_close():
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    driver.switch_to.frame("indexFrame")

def wait_driver(xpath, t=10):
    WebDriverWait(driver, t).until(EC.presence_of_element_located((By.XPATH, xpath)))

def parsing_laws():
    lang_pairs = {"eng": [], "kor": []}
    for i, p in enumerate(driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')):
        if i<4: continue
        for line in p.text.split("\n"):
            if i%2==0:
                lang_pairs["eng"].append(line)
            else:
                lang_pairs["kor"].append(line)

In [ ]:
driver.get(url)

categories = [f'tree_{i}_a' for i in range(2, 47)]
# category loop
base_df = pd.DataFrame()
for j, c in enumerate(categories):
    category = driver.find_element(By.ID, c)
    cate_list = category.text.split(" ")
    cate_name, num_of_laws = cate_list[1], int(cate_list[2].strip('()'))
    driver.execute_script("arguments[0].click();", category)

    # pages
    driver.switch_to.frame('indexFrame')
    # pages = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.paging > a")
    pages = [f'//*[@id="container"]/div/div[3]/a[{k}]' for k in range(3, 13)]
    page_idx = 0
    complete_num = 0
    print(f"{j+1}. {cate_name} 시작...")
    while True:
    # laws loop
        wait_driver('//*[@id="1"]/td[2]')
        laws = driver.find_elements(By.CSS_SELECTOR, '#list1 > tbody > tr > td > p > a')
        page_parsing_num = 0
        for law in laws:
            law_name = law.text.split('\n')[0]
            law.send_keys(Keys.ENTER)

            tab_move()
            wait_driver('/html')

            # Parsing
            eng, kor = [], []
            contents = driver.find_elements(By.CSS_SELECTOR, '#body > div > table > tbody > tr > td')
            for i, c in enumerate(contents[4:-2]):
                if i%2==0:
                    eng.append(c.text)
                else:
                    kor.append(c.text)
            if len(eng)!=len(kor):
                raise Exception(f"Different Length ENG{len(eng)}, KOR{len(kor)}")
            else:
                df = pd.DataFrame({"category":[cate_name]*len(eng), "lawName":[law_name]*len(eng), "ENG":eng, "KOR":kor})
                page_parsing_num+=df.shape[0]
                # print(f"{law_name} 파싱 완료 : {len(eng)} 개")

            # fininsh parsing
            tab_close()
            base_df = pd.concat([base_df, df], axis=0)
        print(f"\t{page_idx+1}페이지 파싱 완료: {page_parsing_num}")
        complete_num += len(laws)
        if complete_num < num_of_laws:
            driver.find_element(By.XPATH, pages[page_idx]).click()
            page_idx+=1
        else:
            break
    driver.get(url)
    wait_driver('//*[@id="tree_1"]')
driver.quit()
